<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/draft_1107.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**패키지 설치**

In [2]:
!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
transformers 4.57.1 requires tokenizers<=0.23.0,>=0.22.0, but you have tokenizers 0.20.3 which is inc

In [3]:
!pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=ccbec0f7f44ce90703ba79a5f6b6b9537f231a820c954771e14f23c48ab92a76
  Stored in directory: /root/.cache/pip/wheels/33/3c/79/b36253689d838af4a0539782853ac3cc38a83a6591ad570dde
Successfully built wikipedia-api


In [35]:
!pip3 install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 66.3 MB/s eta 0:00:00


**upstage api**

In [4]:
# set parameter
UPSTAGE_API_KEY = "up_g7T2cQoLKZH6Oi2n4MHOW706XAdSs"
data_path = "/content/drive/MyDrive/Colab Notebooks/2025_2/nlp" # folder path containing ewah.pdf and samples.csv

**모듈 불러오기**

- Upstage를 활용하는 부분: Embedding & LLM
- Vectorstore: FAISS (임의)
- Chunking: UpstageDocumentParseLoader(Rag_Tutorial과 다른 파싱 모듈임)
- 외부 정보: wikipediaapi (임의)
- Rag chain: RetrievalQA

In [79]:
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_upstage import UpstageDocumentParseLoader
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
import wikipediaapi
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA

**외부 정보 불러오기**

In [50]:
wiki = wikipediaapi.Wikipedia(user_agent= "NLP-RAG/1.0", language = 'en')
page = wiki.page("philosophy")

**Chunking**

In [84]:
text = page.text
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
doc = splitter.create_documents([text])

**Embedding**

In [85]:
upstage_embeddings = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="embedding-passage")

**Vector store에 임베딩**

In [86]:
vectorstore = FAISS.from_documents(doc, upstage_embeddings)

**upstage llm 불러오기**

In [87]:
llm = ChatUpstage(api_key = UPSTAGE_API_KEY)

**prompt template**

In [88]:
prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context.
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {context}
    """
)

**retrieval chain: 검색+생성 과정을 자동으로 처리**

In [92]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) #top 5 문서반환
qa = RetrievalQA.from_chain_type(llm=llm,
                                 retriever=vectorstore.as_retriever(),
                                 chain_type="stuff",
                                 chain_type_kwargs={
                                     "prompt":prompt_template,
                                     "document_variable_name": "context"
                                 },
                                 return_source_documents=True) # 원문출처받기

**쿼리 예시**

In [93]:
query = "How is philosophy defined according to the article"
result = qa({"query": query})

In [94]:
print(result["result"])

Philosophy is a systematic study of general and fundamental questions concerning topics like existence, knowledge, mind, reason, language, and value. It is a rational and critical inquiry that reflects on its methods and assumptions.
